In [2]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-driv/e-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

gpg: keybox '/tmp/tmp462h9lif/pubring.gpg' created
gpg: /tmp/tmp462h9lif/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive
import os
os.chdir('/content/drive')

In [0]:
import os
os.chdir("/content/drive/App/Language_Identification")

In [0]:
f = open('DSLCC-v2.0/train-dev/train.txt','r')
message = f.read()
message = message.splitlines()
f.close()

In [0]:
fraction = 30
timesteps = 50
hidden_units = 768
epochs = 100
batch_size = 10

In [0]:
import re
labels = []
sentences = []
for progress,q in enumerate(message) :
  if progress%fraction == 0 :
    temp = q.split('\t')
    sentences.append(temp[0])
    labels.append(temp[1])

In [0]:
import numpy as np
num = np.zeros(100000)
for sentence in sentences:
  for ch in sentence:
    if num[ord(ch)] == 0 :
      num[ord(ch)] = 1
char_decoding = []
for ch_num,x in enumerate(num) :
  if x == 1 :
    char_decoding.append(ch_num)
char_encoding = {}
for i,ch_num in enumerate(char_decoding) :
  char_encoding[ch_num] = i

In [0]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import numpy as np
sent_num = []
for progress,sentence in enumerate(sentences) :
  temp = []
  for character in sentence[:timesteps] :
    temp.append(char_encoding[ord(character)])
    
  temp = np.asarray(temp)
  input_onehot_encoder = OneHotEncoder(n_values=316,sparse=False)
  temp = temp.reshape(len(temp), 1)
  input_onehot_encoded = input_onehot_encoder.fit_transform(temp)
  
  sent_num.append(input_onehot_encoded)
inputs = np.asarray(sent_num)

In [9]:
from collections import Counter
print(Counter(labels))
print(inputs[0])

Counter({'sk': 601, 'bg': 600, 'bs': 600, 'cz': 600, 'es-AR': 600, 'es-ES': 600, 'hr': 600, 'id': 600, 'mk': 600, 'my': 600, 'pt-BR': 600, 'pt-PT': 600, 'sr': 600, 'xx': 600})
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


In [10]:
#n=4
#ngram = [b[i:i+n] for i in range(len(b)-n+1)]
#print(int(unicode( message[t], "utf-8" )[0]))
print(inputs.shape)

(8401, 50, 316)


In [0]:
label_encoder = LabelEncoder()
label_integer_encoded = label_encoder.fit_transform(labels)

label_onehot_encoder = OneHotEncoder(sparse=False)
label_integer_encoded = label_integer_encoded.reshape(len(label_integer_encoded), 1)
label_hot = label_onehot_encoder.fit_transform(label_integer_encoded)

#inverted = label_encoder.inverse_transform([argmax(onehot_encoded[0, :])])

In [12]:
print(label_hot.shape)

(8401, 14)


In [0]:
from sklearn.model_selection import train_test_split
random_seed = 2
# Split the train and the validation set for the fitting
X_train, X_val, Y_train, Y_val = train_test_split(inputs, label_hot, test_size = 0.1, random_state=random_seed)

In [16]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM

model = Sequential()
model.add(LSTM(hidden_units, input_shape=(timesteps, 316), dropout=0.25, recurrent_dropout=0.45))
model.add(Dense(14, activation = 'softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
print(model.summary())

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 768)               3333120   
_________________________________________________________________
dense_1 (Dense)              (None, 14)                10766     
Total params: 3,343,886
Trainable params: 3,343,886
Non-trainable params: 0
_________________________________________________________________
None


In [17]:
# train
model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, verbose=2)

Epoch 1/100
 - 102s - loss: 2.3655
Epoch 2/100
 - 100s - loss: 2.3387
Epoch 3/100
 - 99s - loss: 2.0177
Epoch 4/100
 - 98s - loss: 1.6426
Epoch 5/100
 - 98s - loss: 1.4741
Epoch 6/100
 - 99s - loss: 1.3874
Epoch 7/100
 - 99s - loss: 1.2937
Epoch 8/100
 - 99s - loss: 1.2388
Epoch 9/100
 - 100s - loss: 1.1948
Epoch 10/100
 - 100s - loss: 1.1440
Epoch 11/100
 - 99s - loss: 1.0842
Epoch 12/100
 - 99s - loss: 1.0414
Epoch 13/100
 - 99s - loss: 0.9990
Epoch 14/100
 - 99s - loss: 0.9649
Epoch 15/100
 - 99s - loss: 0.9417
Epoch 16/100
 - 99s - loss: 0.9061
Epoch 17/100
 - 99s - loss: 0.8762
Epoch 18/100
 - 99s - loss: 0.8407
Epoch 19/100
 - 99s - loss: 0.8357
Epoch 20/100
 - 99s - loss: 0.8141
Epoch 21/100
 - 98s - loss: 0.7929
Epoch 22/100
 - 98s - loss: 0.7732
Epoch 23/100
 - 99s - loss: 0.7518
Epoch 24/100
 - 99s - loss: 0.7350
Epoch 25/100
 - 99s - loss: 0.7259
Epoch 26/100
 - 99s - loss: 0.7038
Epoch 27/100
 - 99s - loss: 0.6894
Epoch 28/100
 - 99s - loss: 0.6721
Epoch 29/100
 - 99s - los

Epoch 54/100
 - 98s - loss: 0.3642
Epoch 55/100
 - 99s - loss: 0.3505
Epoch 56/100
 - 99s - loss: 0.3457
Epoch 57/100
 - 99s - loss: 0.3272
Epoch 58/100
 - 99s - loss: 0.3216
Epoch 59/100
 - 100s - loss: 0.3121
Epoch 60/100
 - 100s - loss: 0.3090
Epoch 61/100
 - 100s - loss: 0.3530
Epoch 62/100
 - 100s - loss: 0.3105
Epoch 63/100
 - 99s - loss: 0.2881
Epoch 64/100
 - 99s - loss: 0.2631
Epoch 65/100
 - 99s - loss: 0.2630
Epoch 66/100
 - 99s - loss: 0.2465
Epoch 67/100
 - 99s - loss: 0.2503
Epoch 68/100
 - 100s - loss: 0.2358
Epoch 69/100
 - 101s - loss: 0.2461
Epoch 70/100
 - 101s - loss: 0.2335
Epoch 71/100
 - 101s - loss: 0.2399
Epoch 72/100
 - 101s - loss: 0.2359
Epoch 73/100
 - 101s - loss: 0.2212
Epoch 74/100
 - 101s - loss: 0.2140
Epoch 75/100
 - 101s - loss: 0.2169
Epoch 76/100
 - 101s - loss: 0.2124
Epoch 77/100
 - 101s - loss: 0.2145
Epoch 78/100
 - 101s - loss: 0.2073
Epoch 79/100
 - 101s - loss: 0.2095
Epoch 80/100
 - 101s - loss: 0.1976
Epoch 81/100
 - 101s - loss: 0.1983
Ep

In [18]:
# evaluate
Y_pred = model.predict(X_val)#, batch_size=n_batch, verbose=0)
Y_pred_classes = np.argmax(Y_pred,axis = 1) 
Y_true = np.argmax(Y_val,axis = 1) 
print(Y_pred_classes.shape)
count = 0
for i in range(len(Y_pred)):
  #print(str(Y_pred_classes[i])+" "+str(Y_true[i]))
  if(Y_pred_classes[i] == Y_true[i]) :
    count+=1
print(str(count)+" "+str(len(Y_pred)))

(841,)
525 841


In [19]:
model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, verbose=2)

Epoch 1/100
 - 99s - loss: 0.1527
Epoch 2/100
 - 99s - loss: 0.1470
Epoch 3/100
 - 99s - loss: 0.1382
Epoch 4/100
 - 99s - loss: 0.1462
Epoch 5/100
 - 100s - loss: 0.1371
Epoch 6/100
 - 101s - loss: 0.1317
Epoch 7/100
 - 100s - loss: 0.1422
Epoch 8/100
 - 100s - loss: 0.1356
Epoch 9/100
 - 101s - loss: 0.1370
Epoch 10/100
 - 100s - loss: 0.1177
Epoch 11/100
 - 100s - loss: 0.1305
Epoch 12/100
 - 100s - loss: 0.1286
Epoch 13/100
 - 101s - loss: 0.1246
Epoch 14/100
 - 101s - loss: 0.1276
Epoch 15/100
 - 102s - loss: 0.1214
Epoch 16/100
 - 102s - loss: 0.1183
Epoch 17/100
 - 102s - loss: 0.1168
Epoch 18/100
 - 102s - loss: 0.1244
Epoch 19/100
 - 102s - loss: 0.1167
Epoch 20/100
 - 101s - loss: 0.1232
Epoch 21/100
 - 101s - loss: 0.1166
Epoch 22/100
 - 101s - loss: 0.1101
Epoch 23/100
 - 101s - loss: 0.1050
Epoch 24/100
 - 101s - loss: 0.1175
Epoch 25/100
 - 101s - loss: 0.1148
Epoch 26/100
 - 101s - loss: 0.1117
Epoch 27/100
 - 102s - loss: 0.1102
Epoch 28/100
 - 101s - loss: 0.1070
Epoch

 - 99s - loss: 0.0860
Epoch 54/100
 - 100s - loss: 0.0822
Epoch 55/100
 - 100s - loss: 0.0794
Epoch 56/100
 - 100s - loss: 0.0820
Epoch 57/100
 - 99s - loss: 0.0873
Epoch 58/100
 - 100s - loss: 0.0865
Epoch 59/100
 - 99s - loss: 0.0830
Epoch 60/100
 - 99s - loss: 0.0744
Epoch 61/100
 - 99s - loss: 0.0809
Epoch 62/100
 - 100s - loss: 0.0818
Epoch 63/100
 - 100s - loss: 0.0822
Epoch 64/100
 - 101s - loss: 0.0758
Epoch 65/100
 - 100s - loss: 0.0688
Epoch 66/100
 - 99s - loss: 0.0740
Epoch 67/100
 - 99s - loss: 0.0812
Epoch 68/100
 - 99s - loss: 0.0810
Epoch 69/100
 - 100s - loss: 0.0687
Epoch 70/100
 - 101s - loss: 0.0756
Epoch 71/100
 - 101s - loss: 0.0767
Epoch 72/100
 - 101s - loss: 0.0714
Epoch 73/100
 - 100s - loss: 0.0750
Epoch 74/100
 - 100s - loss: 0.0707
Epoch 75/100
 - 100s - loss: 0.0720
Epoch 76/100
 - 100s - loss: 0.0704
Epoch 77/100
 - 99s - loss: 0.0733
Epoch 78/100
 - 100s - loss: 0.0785
Epoch 79/100
 - 99s - loss: 0.0688
Epoch 80/100
 - 99s - loss: 0.0654
Epoch 81/100
 - 

In [20]:
# evaluate
Y_pred = model.predict(X_val)#, batch_size=n_batch, verbose=0)
Y_pred_classes = np.argmax(Y_pred,axis = 1) 
Y_true = np.argmax(Y_val,axis = 1) 
print(Y_pred_classes.shape)
count = 0
for i in range(len(Y_pred)):
  #print(str(Y_pred_classes[i])+" "+str(Y_true[i]))
  if(Y_pred_classes[i] == Y_true[i]) :
    count+=1
print(str(count)+" "+str(len(Y_pred)))

(841,)
524 841


In [21]:
!ls "/content/drive/App/Language_Identification"
os.chdir('Models')
model.save("Slide_"+str(hidden_units)+"_"+str(timesteps)+"_"+str(epochs)+"_"+str(batch_size)+"_"+str(fraction)+".h5")

drive	    Models	      Slide_768_100.h5		  SLIDE.ipynb
DSLCC-v2.0  Slide_300_100.h5  Slide_768_30_100_100_10.h5
